<a href="https://colab.research.google.com/github/julio20196/pec2-ae/blob/main/PEC_2_JulioSanchez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

Para estra PEC, he escogido el dataset de `wine.csv`.

**Wine.csv --> definir la calidad (clasificación)**



## Primera parte (20 puntos)
Crear el pipeline de ML (con transformers y estimators) para dar un resultado (los labels están definidos arriba).

---
Primero, cargamos las librerías y el dataset (desde el repositorio):


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [29]:
df = pd.read_csv("https://raw.githubusercontent.com/julio20196/pec2-ae/main/wine.csv")
df.head()

,alcohol,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,quality
0,9.4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,bad
1,9.8,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,bad
2,9.8,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,bad
3,9.8,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,good
4,9.4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,bad


Para este ejercicio he planteado un modelo simple de regresión logística con y lo he incorporado todo como pipeline. La imputación es simple y no hay `OneHot Encoding` puesto que todas las variables *features* son numéricas.

In [30]:
# Dividir los datos en características (X) y etiquetas (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir las columnas numéricas y categóricas (en este caso, no hay categóricas)
numeric_features = X.columns

# Crear transformers para la imputación y la estandarización de características numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Crear la columna Transformer que aplica las transformaciones solo a las características numéricas
preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_features)
])

# Definir el clasificador
classifier = LogisticRegression()

# Crear la pipeline que combina el preprocesamiento y el clasificador
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

Este modelo alcanza una precisión del 74%:

In [31]:
# Entrenar el modelo
pipeline.fit(X_train, y_train)

# Evaluar el modelo
accuracy = pipeline.score(X_test, y_test)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.740625


Podemos guardar los resultados en el dataframe `results`. En este dataframe podemos almacenar las características utilizadas, la etiqueta real y la etiqueta predicha:

In [32]:
# Realizar predicciones en el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Crear un DataFrame con las características de prueba, etiquetas verdaderas y predicciones
results = pd.DataFrame({'Features': X_test.values.tolist(),
                              'True Label': y_test,
                              'Predicted Label': y_pred})

results

,Features,True Label,Predicted Label
803,"[9.6, 7.7, 0.56, 0.08, 2.5, 0.114, 14.0, 46.0,...",good,bad
124,"[9.5, 7.8, 0.5, 0.17, 1.6, 0.082, 21.0, 102.0,...",bad,bad
350,"[9.9, 10.7, 0.67, 0.22, 2.7, 0.107, 17.0, 34.0...",good,good
682,"[9.8, 8.5, 0.46, 0.31, 2.25, 0.078, 32.0, 58.0...",bad,bad
1326,"[10.6, 6.7, 0.46, 0.24, 1.7, 0.077, 18.0, 34.0...",good,good
...,...,...,...
1259,"[11.3, 6.8, 0.64, 0.0, 2.7, 0.123, 15.0, 33.0,...",good,good
1295,"[9.5, 6.6, 0.63, 0.0, 4.3, 0.093, 51.0, 77.5, ...",bad,bad
1155,"[9.8, 8.3, 0.6, 0.25, 2.2, 0.118, 9.0, 38.0, 0...",bad,bad
963,"[11.2, 8.8, 0.27, 0.39, 2.0, 0.1, 20.0, 27.0, ...",good,good


## Segunda Parte (30 puntos)

Empaquetar el algoritmo, dos formas posibles a elección:

1. **Como un archivo de tipo pkl**
2. Como una librería disponible en PiPy.

La primera opción dará 10 puntos menos.

En mi caso he seleccionado la primera opción (en negrita). Para realizar esta tarea, se requiere la librería `pickle`.

In [33]:
import pickle

with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

Podemos comprobar que el archivo .pkl funciona cargando el algoritmo desde el mismo:

In [34]:
df = pd.read_csv("https://raw.githubusercontent.com/jr-marquez/analitica-escalable/main/course/wine.csv")
df.head()

with open('modelo_entrenado.pkl', 'rb') as file:
    trained_model = pickle.load(file)

predictions = trained_model.predict(df)

predictions

array(['bad', 'bad', 'bad', ..., 'good', 'good', 'good'], dtype=object)

Para descargar el archivo .pkl en el PC propio desde GoogleColab, basta con:

In [35]:
from google.colab import files

files.download('modelo_entrenado.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>